In [1]:
"""
Sample SQLAlchemy Python Script for Connecting to GED Testing Services DB
"""

import sqlalchemy
import pandas as pd

"""
Call out for how to connect to SQL Server using 
PyODBC and SQL Server Driver 17 w/ Trusted Connection
"""
def connect_sql(server, database): 
    print("Connecting to Server {} and Database {}".format(server, database))
    db_engine = sqlalchemy.create_engine('mssql+pyodbc://{}/{}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'.format(server, database))
    conn = db_engine.connect()
    return conn


"""
Limited query example using Pandas read_sql function
"""
def get_data():
    # We need the server and database name here to form our SQL connection using function
    conn = connect_sql('localhost','Test3') 
    
    # We need connection formed in previous line and query here
    table_df = pd.read_sql("""
                           SELECT * FROM CANDIDATE as c
                           INNER JOIN TEST_DATA as t
                           ON (c.CANDIDATE_ID=t.CANDIDATE_ID)
                           """, con=conn) 
    return table_df

In [2]:
df = get_data()

Connecting to Server localhost and Database Test3


### Clean-up

__Dropped Columns__
- `LAST_YEAR_INCOME`: It is null for 84% for rows
- `C_ADDRESS`,`DATE_OF_BIRTH`: Encrypted and PII

#### Illogical Values
- Birth Year is limited to 1922: other wise substituted with null
- Dropping Rows where `TEST_CENTER_ID` is null (655 rows, not ged_ready, not on_vue)
- Dropping Rows for `GED_READY` exam (not the actual test)
- Dropping Rows where `C_STATE` or `T_STATE` is null --> Candidates & tests outside the USA
- Dropping Rows where `EXAM_START` or `RESULT_ID` is null.
- What about when score=0?

#### Handling Null values
- `C_COUNTY`: Filling in with "Not Reported"
- `GENDER`: Filling in with "Not Reported".
- `HIGHEST_GRADE_COMPLETED`: Using "Other" as filling it with mode/ordinal median could skew the data .
- `ZIP`: Filling in with "Not Reported"
- `TESTING_REASON`: Filling in with "Not Reported"
- `SCHOOL_INCOMPLETE_REASON`: Filling in with "Not Reported"
- `STUDIED_FOR_GED`: Filling in with "Not Reported"
- `ETHNICITY`: Filling in with "Not Reported". Also fill `RACE_NONE` as True
- `PREP_CENTER`: Filling in with "Not Used"
- `PREP_CENTER_STATE`: Filling in with "Not Used"
- `ENROLLMENT_STATUS`: Filling in with "Unknown"
- `CREDENTIAL_DATE`: Filling in with "No Credentials yet"
- `EXAM_SUBJECT`: Null for 0.03% rows. Seems like its null for exams taken in correctional facilities.
- `RESULT_ID`: 0.023816. Score is null in this scenario.
- `EXAM_START`: 0.057363. If its null, the score is 0. There are also values from 2015-20, when the exam date is earlier to the account setup date --> indicates system migration etc.
- `SCORE`: 0.023816. Scores = 0 could also be considered null. Score is zero even when EXAM_START is not null.
- `FIRST_COMPLETE`:91.5% nulls are for GED_READY tests. Around 7.5% are when the EXAM_START is null. These indicators were created after a while, every value before index 2852819 is null. Not sure how to fill in these values as passign criteria is vague. Values as low as 70 are marked as passed, while many people who have same scores but some have passed but others didn't.
- `FIRST_PASSED`: 0.596066
- `MAKE_COMPLETE`: 0.596066
- `MAKE_PASSED`: 0.596066

In [3]:
# Removing duplicate candidate_id column
df = df.iloc[:,1:]

In [4]:
# Subsetting df to include only valid values of required columns
df = df[df["TEST_CENTER_ID"].notnull()]
df = df[df["C_STATE"].notnull()]
df = df[df["T_STATE"].notnull()]
df = df[df["EXAM_START"].notnull()]
df = df[df["RESULT_ID"].notnull()]
df = df[df["EXAM_SUBJECT"].notnull()]

# DROPPING GED_READY TEST RESULTS
# df = df[df["GED_READY"]==0]

In [6]:
# DROPPING INTERNATIONAL ZIP CODES
# which either contain Alphabets or a hyphen (but not after 5 digits)
# Postal code should have the format: 12345 or 12345-0943
df = df[(df["T_POSTAL_CODE"].str.match("\d{4,}(\-\d{4})?$"))&~(df['JURIS_NAME']=="International")]
# Removing cases when the state is null --> only in international tests, but somehow the juris_name was not "international"
df = df[df['T_STATE'].notnull()]

In [7]:
df.drop(columns=["C_ADDRESS","DATE_OF_BIRTH","LAST_YEAR_INCOME"], inplace=True)#"GED_READY"

In [8]:
col_fill_map = {
"FIRST_COMPLETE": "Not Reported",
"FIRST_PASSED": "Not Reported",
"MAKE_COMPLETE": "Not Reported",
"MAKE_PASSED": "Not Reported",
"ETHNICITY": "Not Reported",
"PREP_CENTER": "Not Used",
"PREP_CENTER_STATE": "Not Used",
"ENROLLMENT_STATUS": "Not Reported",
"CREDENTIAL_DATE": "No Credentials",
"TESTING_REASON": "Not Reported",
"SCHOOL_INCOMPLETE_REASON": "Not Reported",
"STUDIED_FOR_GED": "Not Reported",
"GENDER": "DECLINE",
"C_COUNTY": "Not Reported",
"HIGHEST_GRADE_COMPLETED": "OTHER"
}
for col, fill_value in col_fill_map.items():
    df[col] = df[col].fillna(fill_value)

In [9]:
df["zipcode"] = df["T_POSTAL_CODE"].apply(lambda x: x[:5]).astype(int)

In [10]:
df.to_csv("E:\Group Folder\Data\cleaned_data_backup.csv",index=False)

In [21]:
# possible way to merge
# not merging yet as we don't know which columns to use
# df.merge(acs, on="zipcode")

### Random Explorations

In [58]:
df[(df['SCORE']==0)&(df["CREDENTIAL_DATE"].notnull())]

,ACCOUNT_SETUP_COMPLETE_DATE,C_CITY,C_STATE,C_COUNTY,BIRTH_YEAR,GENDER,HIGHEST_GRADE_COMPLETED,ZIP,JURIS_NAME,JURIS_POSTAL_CODE,...,FIRST_PASSED,MAKE_COMPLETE,MAKE_PASSED,TEST_CENTER_ID,T_NAME,T_STATE,T_ADDRESS,T_CITY,T_POSTAL_CODE,zipcode
52,2021-01-06 00:03:00,Fremont,NE,Dodge,2002,MALE,C_12,68025,Nebraska,NE,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
62,2020-08-30 23:52:00,Tacoma,WA,Pierce,2000,FEMALE,C_11,98445,Washington,WA,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
69,2021-02-18 19:01:00,Portland,OR,Multnomah,1988,MALE,C_12,97201,Oregon,OR,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
194,2021-02-06 17:02:00,Chicago,IL,Cook,2001,MALE,C_12,60619,Illinois,IL,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
202,2020-11-18 22:05:00,Denmark,SC,Bamberg,1977,DECLINE,C_6_8,29042,South Carolina,SC,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852703,2022-02-05 04:05:00,Mowrystown,OH,Highland,1983,FEMALE,C_11,45155,Ohio,OH,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
2852719,2020-07-02 17:33:00,Long Beach,CA,Los Angeles,1988,DECLINE,C_12,90813,California,CA,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
2852720,2022-01-09 17:50:00,Lambert,MS,Quitman,1982,FEMALE,C_9,38643,Mississippi,MS,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437
2852726,2020-08-23 18:23:00,Everett,MA,Middlesex,1996,FEMALE,C_11,02149,Massachusetts,MA,...,Not Reported,Not Reported,Not Reported,66745.0,*UP and OP Delivery ONLY - A-BE*,MN,5601 Green Valley Dr.,Bloomington,55437,55437


In [93]:
# FINDING IF ALLL CASES of exam subject being null is in Prisons etc
df[(df["EXAM_SUBJECT"].isnull())&(df["JURIS_NAME"]=="North Carolina")&(~df["T_NAME"].str.contains("CC"))]

380.0    4
470.0    3
440.0    2
400.0    2
350.0    1
410.0    1
420.0    1
430.0    1
340.0    1
290.0    1
Name: SCORE, dtype: int64

In [22]:
# CAses where the account was setup after the exam
df[df['ACCOUNT_SETUP_COMPLETE_DATE']==pd.Timestamp('2020-01-09 14:15:00')]
df.loc[df["EXAM_START"].dt.year<2020,["ACCOUNT_SETUP_COMPLETE_DATE","EXAM_START"]]

,ACCOUNT_SETUP_COMPLETE_DATE,EXAM_START
626058,2021-08-02 18:14:00,2017-11-29 23:54:00
626074,2020-09-15 06:16:00,2017-12-22 02:00:00
626076,2021-02-23 22:33:00,2017-12-05 22:07:00
626077,2020-04-13 17:23:00,2017-11-27 14:04:00
626078,2020-04-13 17:23:00,2017-11-29 14:51:00
...,...,...
1465353,2021-10-07 17:22:00,2017-09-10 23:19:00
1465354,2021-08-15 14:04:00,2017-09-11 13:10:00
1465355,2020-03-02 17:28:00,2017-09-11 15:20:00
1465356,2021-09-15 15:34:00,2017-09-11 14:14:00


In [101]:
# FIRST PASSED IS NULL FOR FIRST 2952818 rows
df.loc[:2852819,'FIRST_PASSED'].isnull().sum()

1522160

In [15]:
df["HIGHEST_GRADE_COMPLETED"].value_counts(dropna=False)

C_11             602742
C_10             581642
C_12             505375
C_9              323612
NaN              291516
DONT_REMEMBER    166448
C_6_8            155088
NEVER             19003
C_PRE_5           12034
OTHER               305
Name: HIGHEST_GRADE_COMPLETED, dtype: int64

In [24]:
df[df['MAKE_COMPLETE']==1].corr()['SCORE']

CANDIDATE_ID                                        -0.027168
BIRTH_YEAR                                           0.138138
STUDY_HELPFUL_ADULT_EDUCATION_CLASS                 -0.110640
STUDY_HELPFUL_ADULT_EDUCATION_TEACHER               -0.100969
STUDY_HELPFUL_AUDIO_STUDY_MATERIALS                 -0.025587
STUDY_HELPFUL_BOOKS_PRINTED_STUDY_MATERIAL          -0.009737
STUDY_HELPFUL_GED_READY                              0.005489
STUDY_HELPFUL_MATERIALS_MOBILE_APP                   0.029065
STUDY_HELPFUL_ONLINE_COURSE_VIDEO_STUDY_MATERIALS    0.063956
STUDY_HELPFUL_OTHER                                  0.012462
STUDY_HELPFUL_SOCIAL_NETWORKING_WEBSITE              0.002306
STUDY_HELPFUL_TV_STUDY_PROGRAM                      -0.008507
STUDY_LOCATION_TEST_PREPARATION_CENTER              -0.064554
INDIAN_OR_ALASKAN                                   -0.006663
ASIAN                                                0.032521
AFRICAN_AMERICAN                                    -0.137427
RACE_DEC

In [9]:
df['LANGUAGE_CODE'].unique()

array(['ENU', 'ESP', 'ENC'], dtype=object)

In [8]:
df['BIRTH_YEAR'].max(), df['BIRTH_YEAR'].min()

(2006, 1886)

In [6]:
df[df['SCORE']==0]['GED_READY'].sum()

39319

In [7]:
len(df[df['GED_READY']==1])

1554848